# OpenUVF Image Segmentation

Copyright © 2019 Southern Company Services, Inc.  All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
#Import Statements
import cv2 as cv
import os
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import math
import lensfunpy
import core.utils.segmentation_utils as segment
from random import sample
from PIL import Image as im


### Inputs

#### Directories
1. images_dir = string specifiying the path (local or global) to the image directory


#### Module parameters
1. module_orientation = string specifying the orientation of the module. Choices are 'vertical' or ' horizontal' 
2. module_aspect_ratio = float specifying the aspect ratio of the module, specified as ratio of longer side to shorter side


#### Function Settings
1. debug = boolean specifying if debug plots should be generated
2. itmax = int specifying the maximum amount of refinement iterations where the segmentation routine is adjusted to increase accuracy

In [2]:
# Directories
images_dir = 'workspace\module_segmentation\Sample Set\FullSize-few'
images_ext = '.JPG'
output_dir = 'workspace\module_segmentation\Sample Set\Processed/'

# Imaging Parameters 
num_rows = 1
module_orientation = 'horizontal'
module_aspect_ratio = 2

# Camera Distortion Parameters
correct_distortion = True
correct_distortion_mode = 'lensfun'
camera_mnfcr = 'NIKON CORPORATION'
camera_model = 'NIKON D3S'
lens_mnfcr = 'SAMYANG'
lens_model = 'Samyang T-S 24mm f/3.5 ED AS UMC'
image_focal_length = 24.0
image_aperture = 1.4
approx_distance = 3
camera_calibration = None


# Function Settings
debug = True
debug_outputs = True
itmax = 10

### Default Plotting Parameters

In [3]:
mpl.rcParams.keys()
mpl.rcParams['axes.spines.bottom'] = False
mpl.rcParams['axes.spines.left'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False

### Debug Plots

In [4]:
def generateDebugPlots():
        
    #Create figure
    fig = plt.figure(num=f'Image {image_name} Preprocessing', tight_layout=True,figsize=[18, 9.5])
    gs = gridspec.GridSpec(4,5)
    mng = plt.get_current_fig_manager()

    # Axis RGB - Plot base image
    axrgb = fig.add_subplot(gs[0,0])
    plt.imshow(rgb)
    plt.title('RGB')

    # Axis R - Plot base image
    axr = fig.add_subplot(gs[0,1])
    plt.imshow(red) 
    plt.title('Red')

    # Axis G - Plot base image
    axg = fig.add_subplot(gs[0,2])
    plt.imshow(gre) 
    plt.title('Green')

    # Axis B - Plot base image
    axb = fig.add_subplot(gs[0,3])
    plt.imshow(blu) 
    plt.title('Blue')

    # Axis RGB Hist - Plot histograms of the three channels
    axrgbhist = fig.add_subplot(gs[0,4])
    plt.plot(red_hist, 'r-'), plt.plot(gre_hist, 'g-'), plt.plot(blu_hist, 'b-')
    plt.xlabel('Intensity'), plt.ylabel('Count'), plt.title('RGB Histograms')

    # Axis HSV - Plot image in hsv color space
    axhsv = fig.add_subplot(gs[1,0])
    plt.imshow(hsv)
    plt.title('HSV')

    # Axis H - Plot image in hsv color space
    axh = fig.add_subplot(gs[1,1])
    plt.imshow(hue)
    plt.title('Hue')

    # Axis S - Plot image in hsv color space
    axs = fig.add_subplot(gs[1,2])
    plt.imshow(sat)
    plt.title('Saturation')

    # Axis V - Plot image in hsv color space
    axv = fig.add_subplot(gs[1,3])
    plt.imshow(val)
    plt.title('Value (brightness)')

    # Axis Hist - Plot histograms of the three channels
    axhsvhist = fig.add_subplot(gs[1,4])
    plt.plot(hue_hist, 'k-', label = 'Hue')
    plt.plot(sat_hist, 'm-', label = 'Sat')
    plt.plot(val_hist, 'c-', label = 'Val')
    plt.xlabel('Intensity'), plt.ylabel('Count'), plt.title('HSV Histograms'), plt.legend(loc='upper left')

    # Axis Grau - Plot standard grayscale image
    axgray = fig.add_subplot(gs[2,0])
    plt.imshow(gray)
    plt.title('Grayscale')

    # Axis Gray Histogram
    axgrayhist = fig.add_subplot(gs[2,1])
    gray_hist = cv.calcHist([gray], [0], None, [256], [0, 256])
    plt.plot(gray_hist, 'k-')
    plt.title('Grayscale Histogram')

    # Axis RGB min - Plot standard grayscale image
    axgray = fig.add_subplot(gs[2,2])
    plt.imshow(rgb_min)
    plt.title('RGB Min (min of each channel)')

    # Axis RGB min Histogram
    axgrayhist = fig.add_subplot(gs[2,3])
    rgb_min_hist = cv.calcHist([rgb_min], [0], None, [256], [0, 256])
    plt.plot(rgb_min_hist, 'k-')
    plt.title('RGB min Histogram')

    # Axis RGB min edges 
    axhe = fig.add_subplot(gs[3,0])
    plt.imshow(edges_rgb_min)
    plt.title('RGB Min Edges')

    # Axis Gray edges - Plot showing the edges detected in the standard grayscale image
    axhe = fig.add_subplot(gs[3,1])
    plt.imshow(edges_gray)
    plt.title('Grayscale Edges')

    # Axis Hue edges - Plot showing the edge
    axhe = fig.add_subplot(gs[3,2])
    plt.imshow(edges_hue)
    plt.title('Hue Edges')

    # Axis final edges - Plot showing the edge
    axhe = fig.add_subplot(gs[3,3])
    plt.imshow(edges)
    plt.title('Edges')

    # Axis Hue Lines - Plot detected lines 
    axhe = fig.add_subplot(gs[3,4])
    plt.imshow(gray_bgr)
    plt.title('Detected Lines (Hough)')

    #Create Second figure
    fig2 = plt.figure(num=f'Image {image_name} Segmentation', tight_layout=True,figsize=[18, 9.5])
    gs2 = gridspec.GridSpec(2,3)
    mng2 = plt.get_current_fig_manager()

    #Selected representative perspective lines
    axls = fig2.add_subplot(gs2[0,0])
    plt.imshow(gray_sample_lines)
    plt.title('Projective Transform Sample Lines')

    #Vanishing point plot
    axvps = fig2.add_subplot(gs2[0,1])
    plt.imshow(gray_bgr)
    plt.plot(vert_vanish_xs, vert_vanish_ys, 'b.')
    plt.plot(hori_vanish_xs, hori_vanish_ys, 'g.')
    plt.xlim(-10000, 10000), plt.ylim(-10000, 10000)
    plt.title('Vanishing Points')

    #Projective Transform Geometry Calculations Plot
    axptg = fig2.add_subplot(gs2[0,2])
    plt.imshow(gray_sample_box)
    plt.title('Projective Transform Geometry')

    #Projective Transform 
    axptg = fig2.add_subplot(gs2[1,0])
    plt.imshow(transed)
    plt.title('Projective Transform')

    #Update figure and Pause
    plt.draw()
    fig.canvas.manager.window.raise_()
    plt.pause(0.01)

## Image Segmentation

In [5]:
%matplotlib 
# Define list of images in the specified directory (NEEDS EDGE HANDLING)
image_list = os.listdir(images_dir)
image_list = [image_name for image_name in image_list if image_name.endswith(images_ext)]

# Predefined global image processing parameters
resize_rows = 600    

#Processing steps
equalize_hist = False
equalize_method = 'global'
    
    
#iterate through images
image_it = 1
for image_name in image_list:
    
    #Log start time
    time_start = time.time()
    
    #Debug Outputs
    print(f'\nProcessing Image {image_name} ({image_it}):') 
    
    #Define image path
    image_path = images_dir + '/' + image_name
    
    #Load image
    image_original = cv.imread(image_path)
    
    #Calculate basic parameters
    image_rows, image_cols, image_channels = image_original.shape;
    aspect_ratio = image_cols/image_rows
    image_pixels = image_cols * image_rows
    
    #Distortion correction - POSSIBLY IMPLEMENT - May cause more distortion than it corrects
    if correct_distortion:
        
        #Print update statement
        print('   Camera Distortion Correction:')
        
        if correct_distortion_mode == 'lensfun':                 #Corrects using lensfunpy, which uses the lensfun database
            
            #Load camera and lens information
            lensfun_db = lensfunpy.Database()
            camera = lensfun_db.find_cameras(camera_mnfcr, camera_model)[0]
            lens = lensfun_db.find_lenses(camera, lens_mnfcr, lens_model)[0]
            
            
            #Correct distortion
            distortion_modifier = lensfunpy.Modifier(lens, camera.crop_factor, image_cols, image_rows)
            distortion_modifier.initialize(image_focal_length, image_aperture, approx_distance)
            undistorted_coords = distortion_modifier.apply_geometry_distortion()
            undistorted = cv.remap(image_original, undistorted_coords, None, cv.INTER_LANCZOS4)
            
        elif correct_distortion_mode == 'camera_calibration':     #Uses an existing calibration file from calibrate_camera utility
            
            x=y
            
        else:
            print('      Distortion Correction Failed. Check inputs')
            undistorted = image_original
    
    #Resize image to nominal size to increase computation speed(and adjust the anticipated aspect ratio as well)
    image_scalar = resize_rows/image_rows
    resize_cols = round(aspect_ratio * resize_rows)
    image = cv.resize(undistorted, (resize_cols, resize_rows))
    
    if debug:
        print(f'   Aspect Ratio: {aspect_ratio:.2f}')
        print(f'   Total Pixels: {image_pixels}')
    #NEED PREPROCESSING STEP WHICH ADJUSTS THE WHITE BALANCE SOMEHOW - WHEN IMAGING, WB should be constant
    
    #Convert to RGB and isolate channels
    rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    rgb_min = np.amin(rgb, axis=2)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    red = rgb[:,:,0]
    gre = rgb[:,:,1]
    blu = rgb[:,:,2]
    
    #Calculate RGB statistics from the image 
    red_mean = np.mean(red)
    red_hist = cv.calcHist([rgb], [0], None, [256], [0, 256])
    gre_mean = np.mean(gre)
    gre_hist = cv.calcHist([gre], [0], None, [256], [0, 256])
    blu_mean = np.mean(blu)
    blu_hist = cv.calcHist([blu], [0], None, [256], [0, 256])
    print(f'   RGB Means: red = {red_mean:.2f};'\
          f' blue = {blu_mean:.2f}; green = {gre_mean:.2f}')
    
    
    #Convert to HSV and isolate channels
    hsv = cv.cvtColor(rgb, cv.COLOR_RGB2HSV)
    hue = hsv[:,:,0]
    sat = hsv[:,:,1]
    val = hsv[:,:,2]
    
    #Calculate HSV statistics from the image
    #hue = hue.astype(np.float64)
    #hue[hue < 48] = hue + 48
    hue_mean = np.mean(hue)
    hue_hist = cv.calcHist([hue], [0], None, [256], [0, 256])
    sat_mean = np.mean(sat)
    sat_hist = cv.calcHist([sat], [0], None, [256], [0, 256])
    val_mean = np.mean(val)
    val_hist = cv.calcHist([val], [0], None, [256], [0, 256])
    print(f'   HSV Means: hue = {hue_mean:.2f};'\
          f' saturation = {sat_mean:.2f}; value = {val_mean:.2f}')

    #Detect if image is particularly dark 
    
    
    
    #Histogram Equalization
    if equalize_hist and (equalize_method == 'adaptive'):
        clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(9,9))
        rgb_min = clahe.apply(rgb_min)
        gray = clahe.apply(gray)
        hue = clahe.apply(hue)
    elif equalize_hist and (equalize_method == 'global'):
        rgb_min = cv.equalizeHist(rgb_min)
        gray = cv.equalizeHist(gray)
        hue = cv.equalizeHist(hue)
    
    
    #Iteratively refine detected edges in the images
    edges, edge_outputs = segment.edgeDetection(rgb, rgb_min, gray, hue, image_pixels, debug=debug)
    rgb_min_smooth = edge_outputs['rgb_min_smooth']
    gray_smooth = edge_outputs['gray_smooth']
    edges_rgb_min = edge_outputs['edges_rgb_min']
    edges_gray = edge_outputs['edges_gray']
    edges_hue = edge_outputs['edges_hue']
    
    #Detect prominent lines
    print('   Line Detection:')
    max_std = 1                                     #STD cutoff for line deviation
    min_line_length = 100
    max_line_gap = 10
    hough_threshold = 100
    hough_theta = np.pi/1080
    hough_rho = 1
    line_it = 0
    line_repeat = True
    while line_repeat & (line_it < 10):
    
        #Calculate lines with specified parameters
        lines, line_points, line_points_edge, line_angles = segment.lineDetection(edges, hough_rho, hough_theta, \
                                                                                  hough_threshold, min_line_length,\
                                                                                  max_line_gap, resize_rows, resize_cols, \
                                                                                  debug=debug)
        
        #Filter lines 
        vert_inds, vert_lines, vert_angles, hori_inds, hori_lines, hori_angles, reject_inds, reject_lines, line_outputs \
            = segment.filterLines(lines, line_points, line_angles, hough_theta, debug = True)
        
        #Extract additional outputs
        vert_angles_mean = line_outputs['vert_angles_mean']
        vert_angles_std = line_outputs['vert_angles_std']
        hori_angles_mean = line_outputs['hori_angles_mean']
        hori_angles_std = line_outputs['hori_angles_std']
        
        #Determine vanishing points
        vert_vanish_xs = []   #PLACEHOLDER
        vert_vanish_ys = []
        hori_vanish_xs = []
        hori_vanish_ys = []  
        
        #Count number of final lines
        num_lines = len(lines)
        num_hori_lines = len(hori_lines)
        num_vert_lines = len(vert_lines)
        num_lines_filtered = num_lines -(num_hori_lines + num_vert_lines)
        
        #Determine if the line detection procedure should be repeated and strengthen or weaken parameters to achieve this
        max_ct = 250
        min_ct = 5      
        if (num_vert_lines < min_ct) or (num_hori_lines < min_ct):
            line_repeat = True
            min_line_length *= 0.9 
            max_line_gap *= 1.25 
            max_std *= 1.1
        elif (num_vert_lines > max_ct) or (num_hori_lines > max_ct):
            line_repeat = True
            min_line_length = 1.1 * min_line_length
            max_line_gap = 0.9 * max_line_gap
            max_std *= 0.75
        else:
            line_repeat = False
            
        #Output update
        print(f'      ({line_it}): {num_lines} Lines Detected - {num_lines_filtered} Filtered - '\
              f'{num_hori_lines} Horizontal ({hori_angles_mean:.2f} +- {hori_angles_std:.2f} deg.)',\
              f'- {num_vert_lines} Vertical ({vert_angles_mean:.2f} +- {vert_angles_std:.2f} deg.)')
        
        #Iterate iteration counter
        line_it = line_it + 1
        
        #Failure update
        if line_repeat and (line_it == 10):
            print('      Line Detection Failed - Image Rejected')
        

    #Plot lines on the image
    if debug:
        gray_bgr = cv.cvtColor(gray_smooth, cv.COLOR_GRAY2BGR)
        gray_bgr = segment.plotLines(gray_bgr, vert_lines, hori_lines, reject_lines)
        gray_bgr_extended = cv.cvtColor(gray_smooth, cv.COLOR_GRAY2BGR)
        gray_bgr_extended = segment.plotLines(gray, None, None, None, all_lines = line_points_edge)
    
    
    #Plot vanishing points and perspective lines
    #if debug:
    #    gray_vanish_pts = gray_bgr
    #    for v in range(0,len(vert_vanish_xs)):
    #        if not (math.isnan(vert_vanish_xs[v])) and not (math.isnan(vert_vanish_ys[v])):
    #            cv.circle(gray_vanish_pts, (int(vert_vanish_xs[v]), int(vert_vanish_ys[v])), 1, (0,128,255), thickness=-1)
    #    for h in range(0,len(hori_vanish_xs)):
    #        if not (math.isnan(hori_vanish_xs[h])) and not (math.isnan(hori_vanish_ys[h])):
    #            cv.circle(gray_vanish_pts, (int(hori_vanish_xs[h]), int(hori_vanish_ys[h])), 1, (0,255,128), thickness=-1)
            
    #Select 4 representative lines - 2 Horizontal and 2 Vertical
    #vert_lines_sort = np.argsort(vert_angles)
    #vert_lines = vert_lines[vert_lines_sort]
    #hori_lines_sort = np.argsort(hori_angles)
    #hori_lines = hori_lines[hori_lines_sort]

    #PASTE HERE
    
    #Calculate mean projective transform
    print('   Projective Transform:')
    proj_trans, rot_ang, gray_sample_lines, gray_sample_box   = segment.calculateProjectiveTransform(lines, vert_inds, hori_inds, gray_smooth, resize_rows, image_scalar, hough_theta)
    
    #Perform Projective Transform
    transed = cv.warpPerspective(image_original, proj_trans, (image_cols, image_rows))
    
    #Rotate image
    rotate_trans = cv.getRotationMatrix2D(tuple(np.array([image_rows, image_cols])/2), rot_ang, 1.0)
    transed = cv.warpAffine(transed, rotate_trans, (image_cols, image_rows))
    
    #Save image
    transed = cv.cvtColor(transed, cv.COLOR_BGR2RGB)
    output_path = output_dir + image_name
    cv.imwrite(output_path, cv.cvtColor(transed, cv.COLOR_BGR2RGB))
    
    #Log end time and calculate total execution time
    time_end = time.time()
    time_execute = time_end - time_start
    print(f'   Processing time: {time_execute:0.4f} s')
    
    #Generate debug plots
    if debug:
        generateDebugPlots()
        
    #Iteration counter    
    image_it += 1
        
        
        
        
        

    

Using matplotlib backend: Qt5Agg

Processing Image DSC03022.JPG (1):
   Camera Distortion Correction:
   Aspect Ratio: 1.50
   Total Pixels: 12007680
   RGB Means: red = 115.08; blue = 126.89; green = 93.01
   HSV Means: hue = 130.13; saturation = 129.01; value = 141.83
   Edge Detection:
      (0): Ratio = 0.69%
   Line Detection:
      (0): 290 Lines Detected - 223 Filtered - 15 Horizontal (6.33 +- 0.41 deg.) - 52 Vertical (96.19 +- 3.09 deg.)
   Projective Transform:
[array([[ 1.06259020e+00,  1.20646382e-01, -1.56852734e+02],
       [-1.09154065e-02,  1.15209375e+00, -9.19408714e+01],
       [-8.38413804e-06,  6.01273262e-05,  1.00000000e+00]]), array([[ 1.02910072e+00,  1.17314174e-01, -9.92479935e+01],
       [-2.01667397e-02,  1.13968065e+00, -7.03647190e+01],
       [-1.29709952e-05,  5.90044522e-05,  1.00000000e+00]]), array([[ 1.04225636e+00,  1.10202107e-01, -1.08164148e+02],
       [-2.51163535e-02,  1.21868972e+00, -1.86556252e+02],
       [-1.12965251e-05,  5.94138645e-05

NameError: name 'y' is not defined